#Necessary Libraries

In [1]:
#code to put give locally not saving in device but temp file saving and deleting it
import speech_recognition as sr
import time
from googletrans import Translator
from gtts import gTTS
import os
import tempfile
import platform

# Dictionary to map spoken language names to language codes
language_map = {
    'afrikaans': 'af', 'albanian': 'sq', 'arabic': 'ar', 'armenian': 'hy', 'bengali': 'bn', 'bosnian': 'bs',
    'catalan': 'ca', 'chechen': 'ce', 'chichewa'
    : 'ny', 'chinese simplified': 'zh-cn', 'chinese traditional': 'zh-tw',
    'corsican': 'co', 'croatian': 'hr', 'czech': 'cs', 'danish': 'da', 'dutch': 'nl', 'english': 'en', 'esperanto': 'eo',
    'estonian': 'et', 'filipino': 'tl', 'finnish': 'fi', 'french': 'fr', 'frisian': 'fy', 'galician': 'gl', 'georgian': 'ka',
    'german': 'de', 'greek': 'el', 'gujarati': 'gu', 'haitian creole': 'ht', 'hausa': 'ha', 'hawaiian': 'haw', 'hebrew': 'he',
    'hindi': 'hi', 'hungarian': 'hu', 'icelandic': 'is', 'igbo': 'ig', 'indonesian': 'id', 'interlingua': 'ia', 'interlingue': 'ie',
    'inuktitut': 'iu', 'irish': 'ie', 'italian': 'it', 'japanese': 'ja', 'javanese': 'jw', 'kannada': 'kn', 'kazakh': 'kk',
    'khmer': 'km', 'kinyarwanda': 'ki', 'korean': 'ko', 'kurdish': 'ku', 'lao': 'lo', 'latin': 'la', 'latvian': 'lv',
    'lithuanian': 'lt', 'luxembourgish': 'lb', 'macedonian': 'mk', 'malagasy': 'mg', 'malayalam': 'ml', 'maltese': 'mt',
    'maori': 'mi', 'marathi': 'mr', 'myanmar': 'my', 'mongolian': 'mn', 'nepali': 'my', 'norwegian': 'no', 'odia': 'or',
    'pashto': 'ps', 'persian': 'fa', 'polish': 'pl', 'portuguese': 'pt', 'punjabi': 'pa', 'quechua': 'qu', 'romanian': 'ro',
    'russian': 'ru', 'samoan': 'sm', 'sango': 'sg', 'sanskrit': 'sa', 'sardinian': 'sc', 'serbian': 'sr', 'sesotho': 'st',
    'shona': 'sn', 'sindhi': 'sd', 'sinhala': 'si', 'slovak': 'sk', 'slovenian': 'sl', 'somali': 'so', 'spanish': 'es',
    'sundanese': 'su', 'swahili': 'sw', 'swedish': 'sv', 'tagalog': 'tl', 'tajik': 'tg', 'tamil': 'ta', 'telugu': 'te',
    'thai': 'th', 'tigrinya': 'ti', 'tonga': 'to', 'turkish': 'tr', 'ukrainian': 'uk', 'urdu': 'ur', 'vietnamese': 'vi',
    'welsh': 'cy', 'xhosa': 'xh', 'yiddish': 'yi', 'yoruba': 'yo', 'zulu': 'zu'
}

def get_language_code(language_name):
    """Convert spoken language name to language code."""
    language_name = language_name.lower()
    return language_map.get(language_name, None)

def speak_text(text, lang_code):
    """Convert text to speech and play it directly."""
    tts = gTTS(text=text, lang=lang_code)
    with tempfile.NamedTemporaryFile(delete=True) as temp_file:
        tts.save(temp_file.name)
        os.system(f"start {temp_file.name}" if platform.system() == "Windows" else f"afplay {temp_file.name}" if platform.system() == "Darwin" else f"mpg123 {temp_file.name}")

def real_time_speech_recognition(target_language_code):
    """Perform real-time speech recognition and translate text to speech in target language."""
    recognizer = sr.Recognizer()
    translator = Translator()

    with sr.Microphone() as source:
        print("Please wait. Calibrating microphone...")
        recognizer.adjust_for_ambient_noise(source, duration=1)
        print("Microphone calibrated. Start speaking...")

        while True:
            print("Listening...")
            audio = recognizer.record(source, duration=3.5)

            try:
                text = recognizer.recognize_google(audio)
                print(f"Text recognized: {text}")

                # Translate the recognized text
                translated_text = translator.translate(text, dest=target_language_code).text
                print(f"Translated text: {translated_text}")

                # Convert the translated text to speech and speak it
                speak_text(translated_text, target_language_code)

                if "stop program" in text.lower():
                    print("Stop command recognized. Exiting program.")
                    break
            except sr.RequestError as e:
                print(f"Could not request results; {e}")
            except sr.UnknownValueError:
                print("Could not understand the audio")

            time.sleep(0.10)

if __name__ == "__main__":
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        print("Please tell the language in which you want to translate:")
        audio = recognizer.listen(source)
    
    try:
        spoken_language = recognizer.recognize_google(audio)
        print(f"Language spoken: {spoken_language}")
        target_language_code = get_language_code(spoken_language)

        if target_language_code is None:
            print("Unsupported language. Exiting program.")
        else:
            real_time_speech_recognition(target_language_code)
    except sr.RequestError as e:
        print(f"Could not request results; {e}")
    except sr.UnknownValueError:
        print("Could not understand the audio")


Please tell the language in which you want to translate:
Language spoken: German
Please wait. Calibrating microphone...
Microphone calibrated. Start speaking...
Listening...
Text recognized: my name is Mihir Ranjan
Translated text: Mein Name ist Mihir Ranjan
Listening...
Text recognized: tumhara naam kya hai tumhara naam kya hai
Translated text: Wie heißt Ihr Name, wie heißt Ihr Name?
Listening...
Text recognized: how are you doing
Translated text: Wie geht es dir
Listening...
Text recognized: how you can do this to me
Translated text: Wie du mir das antun kannst
Listening...
Text recognized: life is not fair every time we
Translated text: Das Leben ist nicht jedes Mal fair, wenn wir
Listening...
Text recognized: stop program
Translated text: Programm beenden
Stop command recognized. Exiting program.


In [8]:
# app.py
from flask import Flask, request, render_template, jsonify
import speech_recognition as sr
from googletrans import Translator
from gtts import gTTS
import tempfile
import os
import platform

app = Flask(__name__)

# Dictionary to map spoken language names to language codes
language_map = {
    'afrikaans': 'af', 'albanian': 'sq', 'arabic': 'ar', 'armenian': 'hy', 'bengali': 'bn', 'bosnian': 'bs',
    'catalan': 'ca', 'chechen': 'ce', 'chichewa': 'ny', 'chinese simplified': 'zh-cn', 'chinese traditional': 'zh-tw',
    'corsican': 'co', 'croatian': 'hr', 'czech': 'cs', 'danish': 'da', 'dutch': 'nl', 'english': 'en', 'esperanto': 'eo',
    'estonian': 'et', 'filipino': 'tl', 'finnish': 'fi', 'french': 'fr', 'frisian': 'fy', 'galician': 'gl', 'georgian': 'ka',
    'german': 'de', 'greek': 'el', 'gujarati': 'gu', 'haitian creole': 'ht', 'hausa': 'ha', 'hawaiian': 'haw', 'hebrew': 'he',
    'hindi': 'hi', 'hungarian': 'hu', 'icelandic': 'is', 'igbo': 'ig', 'indonesian': 'id', 'interlingua': 'ia', 'interlingue': 'ie',
    'inuktitut': 'iu', 'irish': 'ie', 'italian': 'it', 'japanese': 'ja', 'javanese': 'jw', 'kannada': 'kn', 'kazakh': 'kk',
    'khmer': 'km', 'kinyarwanda': 'ki', 'korean': 'ko', 'kurdish': 'ku', 'lao': 'lo', 'latin': 'la', 'latvian': 'lv',
    'lithuanian': 'lt', 'luxembourgish': 'lb', 'macedonian': 'mk', 'malagasy': 'mg', 'malayalam': 'ml', 'maltese': 'mt',
    'maori': 'mi', 'marathi': 'mr', 'myanmar': 'my', 'mongolian': 'mn', 'nepali': 'my', 'norwegian': 'no', 'odia': 'or',
    'pashto': 'ps', 'persian': 'fa', 'polish': 'pl', 'portuguese': 'pt', 'punjabi': 'pa', 'quechua': 'qu', 'romanian': 'ro',
    'russian': 'ru', 'samoan': 'sm', 'sango': 'sg', 'sanskrit': 'sa', 'sardinian': 'sc', 'serbian': 'sr', 'sesotho': 'st',
    'shona': 'sn', 'sindhi': 'sd', 'sinhala': 'si', 'slovak': 'sk', 'slovenian': 'sl', 'somali': 'so', 'spanish': 'es',
    'sundanese': 'su', 'swahili': 'sw', 'swedish': 'sv', 'tagalog': 'tl', 'tajik': 'tg', 'tamil': 'ta', 'telugu': 'te',
    'thai': 'th', 'tigrinya': 'ti', 'tonga': 'to', 'turkish': 'tr', 'ukrainian': 'uk', 'urdu': 'ur', 'vietnamese': 'vi',
    'welsh': 'cy', 'xhosa': 'xh', 'yiddish': 'yi', 'yoruba': 'yo', 'zulu': 'zu'
}

def get_language_code(language_name):
    """Convert spoken language name to language code."""
    language_name = language_name.lower()
    return language_map.get(language_name, None)

def speak_text(text, lang_code):
    """Convert text to speech and play it directly."""
    tts = gTTS(text=text, lang=lang_code)
    with tempfile.NamedTemporaryFile(delete=True, suffix='.mp3') as temp_file:
        tts.save(temp_file.name)
        if platform.system() == "Windows":
            os.system(f"start {temp_file.name}")
        elif platform.system() == "Darwin":
            os.system(f"afplay {temp_file.name}")
        else:  # Assume Linux
            os.system(f"mpg123 {temp_file.name}")

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/process_audio', methods=['POST'])
def process_audio():
    # Retrieve audio file from the request
    audio_file = request.files['audio'].read()
    target_language_code = request.form['language_code']

    # Initialize recognizer and translator
    recognizer = sr.Recognizer()
    translator = Translator()

    # Use the audio file as the source for input
    audio = sr.AudioData(audio_file, 16000, 2)

    try:
        # Recognize the speech using Google Web Speech API
        text = recognizer.recognize_google(audio)
        print(f"Text recognized: {text}")

        # Translate the recognized text
        translated_text = translator.translate(text, dest=target_language_code).text
        print(f"Translated text: {translated_text}")

        # Convert the translated text to speech and play it
        speak_text(translated_text, target_language_code)

        # Return success response
        return jsonify({'status': 'success', 'translated_text': translated_text})

    except sr.RequestError as e:
        print(f"Could not request results; {e}")
        return jsonify({'status': 'error', 'message': str(e)})
    except sr.UnknownValueError:
        print("Could not understand the audio")
        return jsonify({'status': 'error', 'message': 'Could not understand the audio'})

if __name__ == '__main__':
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat
Traceback (most recent call last):
  File "/Users/doctorranjan/anaconda3/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/Users/doctorranjan/anaconda3/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/Users/doctorranjan/myenv/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/Users/doctorranjan/myenv/lib/python3.10/site-packages/traitlets/config/application.py", line 1074, in launch_instance
    app.initialize(argv)
  File "/Users/doctorranjan/myenv/lib/python3.10/site-packages/traitlets/config/application.py", line 118, in inner
    return method(app, *args, **kwargs)
  File "/Users/doctorranjan/myenv/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 692, in initialize
    self.init_sockets()
  File "/Users/doctorranjan/myenv/lib/python

SystemExit: 1

/Users/doctorranjan/myenv/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [3]:
python app.py


SyntaxError: invalid syntax (2255720966.py, line 1)